# RGB crack detection tests

In [14]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import pandas as pd

from scipy import ndimage as ndi
from skimage.util import img_as_float
from skimage.filters import gabor_kernel

A key way of characterizing the status of the pole is to find whether its surface has cracks, holes, or any other imperfections. A way of doing this is by employing crack detection algorithms. Quite a bunch have been proposed, and some are briefly described in [this survey article](https://www.sciencedirect.com/science/article/pii/S1110016817300236?ref=pdf_download&fr=RR-2&rr=7f6822a8db571c0e)

## Gabor filter

Gabor filters are 2D kernels that can be convolved with an image to extract features from it, by determining if a certain frequecy is present in certain areas of an image, and it is thought to work similarly to mammal visual perception. Among other uses, it can be employed to detect cracks in concrete, as stated by [this article](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6728529). Therefore, it is natural to ask if these can be applied to wood cracks too.

In [95]:
def crop(image):
    y_nonzero, x_nonzero = np.nonzero(image)
    return image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]

def apply_kernel(pole_id, theta, sigma, freq, offset, only_kernel=False):
    
    image_names = []
    images = []
    
    for rotation in [0, 90, 180, 270]:
        img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_masked.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.blur(img, (10,10))
        img = crop(img)

        scale_percent = 25
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)

        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        
        image_names.append(f'Pole_{pole_id}_{rotation}')
        images.append(img)
    
    def compute_feats(image, kernels):
        feats = np.zeros((len(kernels), 2), dtype=np.double)
        for k, kernel in enumerate(kernels):
            filtered = ndi.convolve(image, kernel, mode='wrap')
            feats[k, 0] = filtered.mean()
            feats[k, 1] = filtered.var()
        return feats
    
    '''
    kernels = []
    for theta in range(4):
        theta = theta / 4. * np.pi
        for sigma in (6, 12):
            for frequency in (0.05, 0.25):
                kernel = np.real(gabor_kernel(frequency, theta=theta, sigma_x=sigma, sigma_y=sigma))
                kernels.append(kernel)
    
    features = np.zeros((4, len(kernels), 2), dtype=np.double)
    features[0, :, :] = compute_feats(images[0], kernels)
    features[1, :, :] = compute_feats(images[1], kernels)
    features[2, :, :] = compute_feats(images[2], kernels)
    features[3, :, :] = compute_feats(images[3], kernels)
    '''
    
    def power(image, kernel):
        # Normalize images for better comparison.
        image = (image - image.mean()) / image.std()
        return np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 + ndi.convolve(image, np.imag(kernel), mode='wrap')**2)
    
    # Plot a selection of the filter bank kernels and their responses.

    theta = theta / 4. * np.pi
    kernel = gabor_kernel(freq, theta=theta, sigma_x=sigma, sigma_y=sigma, offset=offset)
    kernel_param = f"theta={theta * 180 / np.pi},\nfrequency={freq:.2f},\nsigma={sigma:.2f},\noffset={offset:.2f}"
    # Save kernel and the power image for each image
    if only_kernel:
        result = (kernel, None)
    else:
        result = (kernel, [power(img, kernel) for img in images])
    
    return result, kernel_param

In [93]:
def plot_kernel(result, param, pole_id, only_kernel=False):
    image_names = []
    images = []
    
    for rotation in [0, 90, 180, 270]:
        img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_masked.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.blur(img, (10,10))
        img = crop(img)

        scale_percent = 25
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)

        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        
        image_names.append(f'Pole_{pole_id}_{rotation}')
        images.append(img)
    
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(12, 6))
    plt.gray()

    fig.suptitle('Image responses for Gabor filter kernels', fontsize=12)

    axes[0][0].axis('off')

    # Plot original images
    for label, img, ax in zip(image_names, images, axes[0][1:]):
        ax.imshow(img)
        ax.set_title(label, fontsize=9)
        ax.axis('off')

    for label, (kernel, powers), ax_row in zip([param], [result], axes[1:]):
        # Plot Gabor kernel
        ax = ax_row[0]
        ax.imshow(np.real(kernel))
        ax.set_ylabel(label, fontsize=7)
        ax.set_xticks([])
        ax.set_yticks([])

        # Plot Gabor responses with the contrast normalized for each filter
        if not only_kernel:
            vmin = min([np.min(power) for power in powers])
            vmax = max([np.max(power) for power in powers])
            for patch, ax in zip(powers, ax_row[1:]):
                ax.imshow(patch, vmin=vmin, vmax=vmax)
                ax.axis('off')

    param_clean = param.replace(",\n","_")
    plt.savefig(f'../Figures/Gabor/{pole_id}_gabor_{param_clean}.png')
    plt.close()

In [ ]:
for pole_id in ['Ny', 5, 6, 30, 41]:
    kernels, params = apply_kernels(pole_id)
    plot_kernels(kernels, params, pole_id)

As it can be seen in the images, Gabor filters are an extremely promising way of charaterizing the surface of a pole, being able to capture information regarding the smoothness of the surface, the presence of cracks, and more phenomena. However, not all filters tested here are useful, and some are redundant. Furthermore, there are some pole characteristics that are not yet captured by the filters, like the hole in pole 41, rotated 180 degrees. It is therefore a good option to pick the seemingly best filters from this step, and add more that can capture these characteristics. More specifically:

  - Filters with $\sigma = 6$ will be discarded, since these are too small for the pole images. This also means that the final Gabor filters will have to be designed having the image size in mind. Another option is to resize the pole images to be     approximately the same size as these.
  - New filters with $frequency = 0.01$ will be added, in order to capture larger relevant areas, such as the aforementioned hole. Since cracks tend to appear as dark areas in the middle of brighter areas, this kernel should have an offset of 
    $\pi$ in order to capture this.
    
In order to ease filter development, the widget below can be used to visualize a filter with given parameters.

In [96]:
kernel_params = [
    (0, 12, 0.01, np.pi),
    (0, 12, 0.05, 0),
    (0, 12, 0.25, 0),
    (1, 12, 0.01, np.pi),
    (1, 12, 0.05, 0),
    (1, 12, 0.25, 0),
    (2, 12, 0.01, np.pi),
    (2, 12, 0.05, 0),
    (2, 12, 0.25, 0),
    (3, 12, 0.01, np.pi),
    (3, 12, 0.05, 0),
    (3, 12, 0.25, 0),
]

for pole_id in ['Ny', 5, 6, 30, 41]:
    for param in kernel_params:
        result, param = apply_kernel(pole_id, *param)
        plot_kernel(result, param, pole_id)

After visualizing the results of some filters, and after considering the poles and the shapes of cracks and holes, I have opted for creating this series of filters. These begin from full-resolution filters at 1000x1000, and get re-scaled to detect these features at various levels of detail.

- Vertical holes: 500x1000 sized filter, $\sigma = 100$, $\theta = 0$, $\lambda = 1$, $\gamma = 0.4$, $\psi = \pi$
- Horizontal holes: like the previous one, but rotated 90º
- Big cracks: 500x1000 sized filter, $\sigma = 100$, $\theta = 0$, $\lambda = 1$, $\gamma = 0$, $\psi = \pi$
- Thinner cracks: 500x1000 sized filter, $\sigma = 25$, $\theta = 0$, $\lambda = 1$, $\gamma = 0$, $\psi = \pi$
- Thinnest cracks: 500x1000 sized filter, $\sigma = 3$, $\theta = 0$, $\lambda = 1$, $\gamma = 0$, $\psi = \pi$

In [172]:
import imutils

def apply_custom_kernels(pole_id, only_kernels=False):
    image_names = []
    images = []
    
    max_width = -1
    
    for rotation in [0, 90, 180, 270]:
        img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_masked.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.blur(img, (10,10))
        img = crop(img)

        scale_percent = 15
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        max_width = max(max_width, width)

        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        
        image_names.append(f'Pole_{pole_id}_{rotation}')
        images.append(img)
            
    def power(image, kernel):
        # Normalize images for better comparison.
        image = (image - image.mean()) / image.std()
        return ndi.convolve(ndi.convolve(image, np.real(kernel), mode='wrap')**2 + ndi.convolve(image, np.imag(kernel), mode='wrap')**2)
    
    kernels = []
    
    # Hole detection kernels
    vertical_base_kernel = cv2.getGaborKernel((500, 1000), 100, 0, 1, 0.4, np.pi)
    half_width_vertical_kernel = imutils.resize(vertical_base_kernel, width=max_width // 2)                      # For detecting holes of different sizes
    quarter_width_vertical_kernel = imutils.resize(vertical_base_kernel, width=max_width // 4)
    eigth_width_vertical_kernel = imutils.resize(vertical_base_kernel, width=max_width // 8)
    kernels += [half_width_vertical_kernel, quarter_width_vertical_kernel, eigth_width_vertical_kernel]
    kernels += [half_width_vertical_kernel.T, quarter_width_vertical_kernel.T, eigth_width_vertical_kernel.T]   # Also add rotated versions of these hole detectors
    
    # Crack detection kernels
    #big_crack_kernel = cv2.getGaborKernel((300, 5), 100, 0, 1, 0, np.pi)
    #medium_crack_kernel = cv2.getGaborKernel((300, 5), 25, 0, 1, 0, np.pi)
    #small_crack_kernel = cv2.getGaborKernel((300, 5), 3, 0, 1, 0, np.pi)
    kernels += [imutils.resize(big_crack_kernel, width=max_width // 2), imutils.resize(medium_crack_kernel, width=max_width // 2), imutils.resize(small_crack_kernel, width=max_width // 2)]
    
    kernel_names = [
        'Big hole detector, vertical',
        'Medium hole detector, vertical',
        'Small hole detector, vertical',
        'Big hole detector, horizontal',
        'Medium hole detector, horizontal',
        'Small hole detector, horizontal'
        #'Big crack detector',
        #'Medium crack detector',
        #'Small crack detector',
    ]
        
    results = []
    for kernel in kernels:
        # Save kernel and the power image for each image
        if only_kernels:
            results.append((kernel, None))
        else:
            results.append((kernel, [power(img, kernel) for img in images]))

    return results, kernel_names

In [169]:
def plot_custom_kernels(results, names, pole_id, only_kernels=False):
    image_names = []
    images = []
    
    for rotation in [0, 90, 180, 270]:
        img = cv2.imread(f'../Data/RGB/crops/{pole_id}_{rotation}_crop_masked.jpg')
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.blur(img, (10,10))
        img = crop(img)

        scale_percent = 25
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)

        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        
        image_names.append(f'Pole_{pole_id}_{rotation}')
        images.append(img)
    
    fig, axes = plt.subplots(nrows=17, ncols=5, figsize=(20, 60))
    plt.gray()

    fig.suptitle('Image responses for Gabor filter kernels', fontsize=12)
    print(len(results[0]), len(names))

    axes[0][0].axis('off')

    # Plot original images
    for label, img, ax in zip(image_names, images, axes[0][1:]):
        ax.imshow(img)
        ax.set_title(label, fontsize=9)
        ax.axis('off')

    for label, (kernel, powers), ax_row in zip(names, results, axes[1:]):
        # Plot Gabor kernel
        ax = ax_row[0]
        ax.imshow(np.real(kernel))
        ax.set_ylabel(label, fontsize=7)
        ax.set_xticks([])
        ax.set_yticks([])

        # Plot Gabor responses with the contrast normalized for each filter
        if not only_kernels:
            vmin = min([np.min(power) for power in powers])
            vmax = max([np.max(power) for power in powers])
            for patch, ax in zip(powers, ax_row[1:]):
                ax.imshow(patch, vmin=vmin, vmax=vmax)
                ax.axis('off')
                
    plt.savefig(f'../Figures/Gabor/{pole_id}_gabor.png')

In [ ]:
for pole_id in ['Ny', 5, 6, 30, 41]:
    kernels, names = apply_custom_kernels(pole_id)
    plot_custom_kernels(kernels, names, pole_id)

After trying all these ever-so-computationally-expensive Gabor kernels (damn how much I wish I had a GPU now...), I think the best way to proceed is with the first set of kernels, together with the hole detectors (it seems like these are producing good activations for the hole in pole 41?)

Furthermore, to perform an even better comparison, this time I will plot the activation histograms, to compare how different images activate in response to different filters.

In [17]:
# Skimage version 0.19.3
# OpenCV version 4.6.0

kernel_params = [
    ('skimage', (0, 12, 0.01, 0)),
    ('skimage', (0, 12, 0.05, 0)),
    ('skimage', (0, 12, 0.25, 0)),
    ('skimage', (1, 12, 0.01, 0)),
    ('skimage', (1, 12, 0.05, 0)),
    ('skimage', (1, 12, 0.25, 0)),
    ('skimage', (2, 12, 0.01, 0)),
    ('skimage', (2, 12, 0.05, 0)),
    ('skimage', (2, 12, 0.25, 0)),
    ('skimage', (3, 12, 0.01, 0)),
    ('skimage', (3, 12, 0.05, 0)),
    ('skimage', (3, 12, 0.25, 0)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 2, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 4, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 8, False)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 2, True)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 4, True)),
    ('cv2', ((500, 1000), 100, 0, 1, 0.4, np.pi, 8, True)),
]

def crop(image):
    y_nonzero, x_nonzero = np.nonzero(image)
    return image[np.min(y_nonzero):np.max(y_nonzero), np.min(x_nonzero):np.max(x_nonzero)]

def power(image, kernel):
        # Normalize images for better comparison.
        image = (image - image.mean()) / image.std()
        return np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 + ndi.convolve(image, np.imag(kernel), mode='wrap')**2)

def plot_activations_kernel_pole_rotation(backend, params, kernel_id, draw=True):
    if draw:
        fig, axes = plt.subplots(5, 8, figsize=(20,16))
    
    # Generate Gabor kernel depending on specified backend
    if backend == 'skimage':
        theta = params[0] / 4. * np.pi
        kernel = gabor_kernel(params[2], theta=theta, sigma_x=params[1], sigma_y=params[1], offset=params[3])
    elif backend == 'cv2':
        kernel_params, inv_scale, transpose = params[:-2], params[-2], params[-1]
        kernel = cv2.getGaborKernel(*kernel_params)
        if transpose:
            kernel = kernel.T
            
    # Compute kernel statistics for all pole figures
    pole_ids = ['Ny',30,41,6,5]
    rotations = [0,90,180,270]
    pole_status = ['31/37', '17/24', '15/24', '12/24', '11/24']
    specific_status = ['New', 'Rotten', 'Cracks', 'Cracks', 'Rotten']
    
    # Prepare dictionaries to store features
    data = {
        'id': [0,30,41,6,5],
        f'kernel_{kernel_id}_mean_0_degrees': [],
        f'kernel_{kernel_id}_std_0_degrees': [],
        f'kernel_{kernel_id}_mode_0_degrees': [],
        f'kernel_{kernel_id}_mean_90_degrees': [],
        f'kernel_{kernel_id}_std_90_degrees': [],
        f'kernel_{kernel_id}_mode_90_degrees': [],
        f'kernel_{kernel_id}_mean_180_degrees': [],
        f'kernel_{kernel_id}_std_180_degrees': [],
        f'kernel_{kernel_id}_mode_180_degrees': [],
        f'kernel_{kernel_id}_mean_270_degrees': [],
        f'kernel_{kernel_id}_std_270_degrees': [],
        f'kernel_{kernel_id}_mode_270_degrees': [],
    }
    
    for pole_idx, pole in enumerate(pole_ids):
        for rotation_idx, rotation in enumerate(rotations):
            # Read, crop, and rescale image
            img = cv2.imread(f'../Data/RGB/crops/{pole}_{rotation}_crop_masked.jpg')
            mask = cv2.imread(f'../Data/RGB/crops/{pole}_{rotation}_crop_mask.jpg')[:,:,0]
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = cv2.blur(img, (10,10))
            img = crop(img)
            mask = crop(mask)
            scale_percent = 15
            width = int(img.shape[1] * scale_percent / 100)
            height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            mask = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
            
            # Properly rescale kernel if it is from OpenCV
            if backend == 'cv2':
                kernel = imutils.resize(kernel, width = width // inv_scale)
            
            
            activations = np.real(power(img, kernel))
            activations_masked = np.where(mask == 255, activations, 0)
            activations_flat = activations_masked.flatten()
            remapped_activations = np.interp(activations_flat, (activations_flat.min(), activations_flat.max()), (0, +1))
            filtered_activations = remapped_activations[remapped_activations > 0.03]    # Remove values near 0 to remove background effects
            n, bins, patches = axes[pole_idx, rotation_idx*2].hist(filtered_activations, bins=100, range=(0,1))
            mode_index = n.argmax()
            mode = (bins[mode_index] + bins[mode_index+1]) / 2
            axes[pole_idx, rotation_idx*2].clear()
            
            bp = axes[pole_idx, rotation_idx*2].boxplot(filtered_activations, 0, '')
            axes[pole_idx, rotation_idx*2].set_xticks([])
            axes[pole_idx, rotation_idx*2].set_ylim([-0.1,1.1])
            axes[pole_idx, rotation_idx*2+1].imshow(activations_masked)
            axes[pole_idx, rotation_idx*2+1].set_xticks([])
            axes[pole_idx, rotation_idx*2+1].set_yticks([])

            # Print mean and standard deviation to compare plots
            mean, stdev, median = np.mean(filtered_activations), np.std(filtered_activations), np.median(filtered_activations)
            for i, line in enumerate(bp['medians']):
                x, y = line.get_xydata()[1]
                text = ' μ={:.2f}\n σ={:.2f}\nmedian={:.2f}\nmode={:.2f}'.format(mean, stdev, median, mode)
                axes[pole_idx, rotation_idx*2].annotate(text, xy=(x, y), fontsize=7)
                
            data[f'kernel_{kernel_id}_mean_{rotation}_degrees'].append(mean)
            data[f'kernel_{kernel_id}_std_{rotation}_degrees'].append(stdev)
            data[f'kernel_{kernel_id}_mode_{rotation}_degrees'].append(mode)
                            
                
    fig.suptitle('Gabor filter activation histograms')
    [axes[0,i*2].set_title(f'{i*90} degrees') for i in range(4)]
    [axes[i,0].set_ylabel(f'Pole {pole}\nStatus: {pole_status[i]}\nCause: {specific_status[i]}', labelpad=60, fontdict={'rotation':0}) for i, pole in enumerate(pole_ids)]
    plt.savefig(f'../Figures/Gabor/boxplots/kernel_{kernel_id}_boxplots.png')
    plt.close()
    fig, axes = plt.subplots()
    axes.imshow(np.real(kernel), cmap='gray')
    plt.savefig(f'../Figures/Gabor/boxplots/kernel_{kernel_id}.png')
    plt.close()
    
    return data

In [ ]:
with plt.ioff():
    for kernel_id, params in enumerate(kernel_params[12:]):
        data = plot_activations_kernel_pole_rotation(*params, kernel_id+12)
        df_kernel = pd.DataFrame(data)
        df_kernel.to_csv(f'../Features/gabor_kernel_{kernel_id+12}.csv')